# Import Libraries

In [46]:
# General system libraries
import os
import sys
from IPython.display import Image, Markdown, display
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Dataframe libraries
import pandas as pd
from pandas import DataFrame, read_csv

# Number manipulation
import scipy.sparse
from scipy.ndimage.filters import generic_filter
import patsy
import numpy as np

# Plotting libaries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline


# Data type libaries
from datetime import datetime as dt

# File manipulation
import pickle
import pandas.io.sql as pd_sql
from sqlalchemy import create_engine
import psycopg2 as pg
from flatten_json import flatten

# NLP libraries
import wikipedia as wiki
from nltk import word_tokenize, sent_tokenize,FreqDist, pos_tag
from nltk.corpus import stopwords
import gensim as gn
from gensim import corpora, models, similarities
from collections import defaultdict
from six import iteritems
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS
import string
import emoji
# import enchant
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer


# Scraping libraries
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
# from scraping_functions.tumblr_api import get_client
# import pytumblr
from google_images_download import google_images_download

# Stats libaries
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import datasets, linear_model, metrics
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import svm, datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.neighbors import KNeighborsClassifier

# Data Storage
from boto.s3.connection import S3Connection
from boto.s3.key import Key

# Other libaries
import geopy

# Define a function to scrape image links

# Define a function to download images from Google using search term

Want to pass in the list of hashtags we obtained from Project 4 as the search terms for scraping Google images. We can also search by general search criteria

In [44]:
def download_google_images(list_of_search_terms, num_images_per_search, output_directory): 
    """
    This method takes a list of search terms and downloads the specified number of images that result from a 
    Google search of that term. 
    :param 
    :return: A dictionary of image filenames and urls 
    """
    response = google_images_download.googleimagesdownload()
    list_of_paths_dicts = []
    for search_term in list_of_search_terms:
        arguments = {"keywords": search_term, "limit": num_images_per_search, "print_urls": True, "chromedriver": "chromedriver", "output_directory": output_directory, "--related_images": None}
        paths_dict = response.download(arguments)
        list_of_paths_dicts.append(paths_dict)
        print(paths_dict)
    return(list_of_paths_dicts)

In [45]:
download_google_images(['anorexia', 'bulimia'], 2, 'test')


Item no.: 1 --> Item name = anorexia
Evaluating...
Starting Download...
Image URL: https://i.ytimg.com/vi/5aEATKztSgs/maxresdefault.jpg
Completed Image ====> 1. maxresdefault.jpg
Image URL: https://images.medicaldaily.com/sites/medicaldaily.com/files/styles/headline/public/2017/03/30/anorexia.png
Completed Image ====> 2. anorexia.png

Errors: 0

{'anorexia': ['/home/victoria/projects/at-risk-id-images/notebooks_and_data/test/anorexia/1. maxresdefault.jpg', '/home/victoria/projects/at-risk-id-images/notebooks_and_data/test/anorexia/2. anorexia.png']}

Item no.: 1 --> Item name = bulimia
Evaluating...
Starting Download...
Image URL: https://www.healthtopia.net/sites/default/files/bulimia.png
Completed Image ====> 1. bulimia.png
Image URL: https://www.wikihow.com/images/thumb/e/e4/Diagnose-Bulimia-Nervosa-Step-3.jpg/aid8073692-v4-728px-Diagnose-Bulimia-Nervosa-Step-3.jpg
Completed Image ====> 2. aid8073692-v4-728px-diagnose-bulimia-nervosa-step-3.jpg

Errors: 0

{'bulimia': ['/home/victo

[{'anorexia': ['/home/victoria/projects/at-risk-id-images/notebooks_and_data/test/anorexia/1. maxresdefault.jpg',
   '/home/victoria/projects/at-risk-id-images/notebooks_and_data/test/anorexia/2. anorexia.png']},
 {'bulimia': ['/home/victoria/projects/at-risk-id-images/notebooks_and_data/test/bulimia/1. bulimia.png',
   '/home/victoria/projects/at-risk-id-images/notebooks_and_data/test/bulimia/2. aid8073692-v4-728px-diagnose-bulimia-nervosa-step-3.jpg']}]

In [35]:
type(paths)

dict

# Prepare S3 buckets

In [ ]:
def create_s3_bucket(bucket_name):
    '''
    Create an S3 bucket from the bucket name.
    param: bucket name
    '''
    conn = S3Connection(os.environ['AWS_ACCESS_KEY'], os.environ['Ahttps://spring-chi18-metis.slack.com/messagesWS_SECRET_KEY'])
    bucket = conn.create_bucket(bucket_name)
    return bucket

In [ ]:
def get_s3_bucket(bucket_name):
    '''
    Get an S3 bucket from the bucket name.
    param: bucket name
    '''
    conn = S3Connection(os.environ['AWS_ACCESS_KEY'], os.environ['AWS_SECRET_KEY'])
    bucket = conn.get_bucket(bucket_name)
    return bucket

# Define a function to upload image files to S3 bucket

In [ ]:
def add_object_to_bucket(bucket, key_name, object_to_upload):
    '''
    Add an object to an S3 bucket.
    param: bucket: the s3 bucket object
    param: key_name: a string that represents the key name that will be stored
    param: object: the python object that you want to assign to that key
    '''
    pickle.dump(object_to_upload, open("/Users/zachariahmiller/temp.p", "wb"))
    k = Key(bucket)
    k.key = key_name
    k.set_contents_from_filename("downloads/{}".format())
    return None

# Define a function to obtain images from S3 bucket

In [38]:
def get_object_from_bucket(bucket, key_name):
    '''
    Add an object to an S3 bucket.
    param: bucket: the s3 bucket object
    param: key_name: a string that represents the key name that will be stored

    returns: object from s3 bucket
    '''
    k = Key(bucket)
    k.key = key_name
    k.get_contents_to_filename("/Users/zachariahmiller/temp.p")
    new_object = pickle.load(open("/Users/zachariahmiller/temp.p", "rb"))
    return new_object

# Define a function to view and label images in notebook

In [ ]:
def label_images()


listOfImageNames = ['/path/to/images/1.png',
                    '/path/to/images/2.png']

for imageName in listOfImageNames:
    display(Image(filename=imageName))

# Summary

### What I did:
1. Defined a function to 